In [1]:
from deepview import DeepView
import numpy as np
import time
import torch
import os
# ---------------------------
import demo_utils as demo

%load_ext autoreload
%autoreload 2
%matplotlib qt

In [2]:
# matplotlib qt seems to be a bit buggy with notebooks, so we execute it multiple times
%matplotlib qt

## Load CIFAR10 and a some models

This notebook tests the DeepView framework on different classifiers

 * ResNet-20 on CIFAR10
 * DecisionTree on MNIST
 * RandomForest on MNIST
 * KNN on MNIST

In [3]:
# device will be detected automatically
# Set to 'cpu' or 'cuda:0' to set the device manually
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

cifar_X, cifar_y = demo.make_cifar_dataset()
resnet20 = demo.create_trained_resnet20(device)

digits_X, digits_y = demo.make_digit_dataset()
decision_tree = demo.create_decision_tree(digits_X, digits_y, max_depth=10)
random_forest = demo.create_random_forest(digits_X, digits_y, n_estimators=100)
kn_neighbors = demo.create_kn_neighbors(digits_X, digits_y, k=10)

Files already downloaded and verified


RuntimeError: CUDA error: all CUDA-capable devices are busy or unavailable

## Usage Instructions

 1. Create a wrapper funktion like ```pred_wrapper``` which receives a numpy array of samples and returns according class probabilities from the classifier as numpy arrays
 2. Initialize DeepView-object and pass the created method to the constructor
 3. Run your code and call ```add_samples(samples, labels)``` at any time to add samples to the visualization together with the ground truth labels.
    * The ground truth labels will be visualized along with the predicted labels
    * The object will keep track of a maximum number of samples specified by ```max_samples``` and it will throw away the oldest samples first
 4. Call the ```show``` method to render the plot

The following parameters must be specified on initialization:


| <p align="left">Variable    | <p align="left">Meaning             |
|----------------------|-------------------|
| <p align="left">```batch_size```    | <p align="left">Batch size to use when calling the classifier |
| <p align="left">```pred_wrapper```    | <p align="left">To enable DeepView to call the classifier |
| <p align="left">```max_samples```      | <p align="left">The maximum amount of samples that DeepView will keep track of |
| <p align="left">```data_shape```         | <p align="left">Shape of the input data (complete shape; for images, include the channel dimension) |
| <p align="left">```n```     | <p align="left">Number of interpolations for distance calculation of two images. |
| <p align="left">```lam```     | <p align="left">Weights the euclidian distance-component against the discriminative fisher distance. A high lambda puts emphasis on the euclidian distance, so structural information of the data will be more relevant for distance computation. |
| <p align="left">```resolution```       | <p align="left">x- and y- Resolution of the decision boundary plot |
| <p align="left">```cmap```             | <p align="left">Name of the colormap that should be used in the plots. |

## Demo with Torch model

In [ ]:
# softmax operation to use in pred_wrapper
softmax = torch.nn.Softmax(dim=-1)

# this is the prediction wrapper, it encapsulates the call to the model
# and does all the casting to the appropriate datatypes
def pred_wrapper(x):
    with torch.no_grad():
        x = np.array(x, dtype=np.float32)
        x = np.transpose(x, [0, 3, 1, 2])
        tensor = torch.from_numpy(x).to(device)
        logits = resnet20(tensor)
        probabilities = softmax(logits).cpu().numpy()
    return probabilities

# the classes in the dataset to be used as labels in the plots
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# --- Deep View Parameters ----
batch_size = 1024
max_samples = 500
data_shape = (32, 32, 3)
n = 3
lam = .64
resolution = 100
cmap = 'tab10'
title = 'ResNet-20 - CIFAR10'

deepview = DeepView(pred_wrapper, classes, max_samples, batch_size, 
                    data_shape, n, lam, resolution, cmap, title=title)

In [8]:
n_samples = 150
sample_ids = np.random.choice(len(cifar_X), n_samples)
X = np.array([ cifar_X[i] for i in sample_ids ])
Y = np.array([ cifar_y[i] for i in sample_ids ])

t0 = time.time()
deepview.add_samples(X, Y)
deepview.show()


print('Time to calculate visualization for %d samples: %.2f sec' % (n_samples, time.time() - t0))

Time to calculate visualization for 150 samples: 29.42 sec


## Add new samples to the visualization

In [21]:
n_new = 200

sample_ids = np.random.choice(len(cifar_X), n_new)
X = np.array([ cifar_X[i] for i in sample_ids ])
Y = np.array([ cifar_y[i] for i in sample_ids ])

t0 = time.time()
deepview.add_samples(X, Y)
deepview.show()

print('Time to add %d samples to visualization: %.2f sec' % (n_new, time.time() - t0))

Distance calculation 20.00 %
Distance calculation 40.00 %
Distance calculation 60.00 %
Distance calculation 80.00 %
Distance calculation 100.00 %
Embedding samples ...


/home/luca/Downloads/umap/umap/umap_.py:1428: UserWarning: Using precomputed metric; transform will be unavailable for new data
  warn("Using precomputed metric; transform will be unavailable for new data")


Computing decision regions ...
Time to add 200 samples to visualization: 82.53 sec


### Example output

As the plot is updatable, it is shown in a separate Qt-window. With the CIFAR-data and the model loaded above, the following plot was produced after 200 samples where added:

**Hyperparameters:**
n = 10
lam = 0.2
resolution = 100

![sample_plot](https://user-images.githubusercontent.com/30961397/72370639-fbab6f00-3702-11ea-98f4-0dc7335777fc.png)

## Tuning the $\lambda$-Hyperparameter

> The $\lambda$-Hyperparameter weights the euclidian distance component.
> When the visualization doesn't show class-clusters, **try a smaller lambda** to put more emphasis on the discriminative distance component that considers the class.
> A smaller $\lambda$ will pull the datapoints further into their class-clusters.
> Therefore, a **too small $\lambda$** can lead to collapsed clusters that don't represent any structural properties of the datapoints. Of course this behaviour also depends on the data and how well the label corresponds to certain structural properties.

Due to separate handling of euclidian and class-discriminative distances, the $\lambda$ parameter can easily be adjusted. Distances don't need to be recomputed, only the embeddings and therefore also the plot of the decision boundary.

In [10]:
deepview.set_lambda(.7)
deepview.show()

Embedding samples ...


/home/luca/Downloads/umap/umap/umap_.py:1428: UserWarning: Using precomputed metric; transform will be unavailable for new data
  warn("Using precomputed metric; transform will be unavailable for new data")


Computing decision regions ...


## Compare performance

For this test, DeepView was run on a GPU (GTX 1060 6GB).
Adding samples may be a bit more time consuming, then just running DeepView on the desired amount of samples to be visualized. This is because the decision boundaries must be calculated twice with a similar time complexity. However, the step of adding 100 samples to 100 existing samples takes less time then computing it from scratch for 200 samples. This is because distances were already computed for half of the samples and can be reused.

| <p align="left">Szenario | Time |
| -------- | ---- |
| <p align="left">From scratch for 100 samples | 31.20 sec |
| <p align="left">Adding 100 samples (100 already added) | 66.89 sec |
| <p align="left">From scratch for 200 samples | 71.16 sec |
| <p align="left">200 samples when adding 100 samples in two steps | 98.19 sec |

In [52]:
deepview.reset()

n_samples = 200
sample_ids = np.random.choice(len(cifar_X), n_samples)
X = np.array([ cifar_X[i] for i in sample_ids ])
Y = np.array([ cifar_y[i] for i in sample_ids ])

t0 = time.time()
deepview.add_samples(X, Y)
deepview.show()

print('Time to calculate visualization for %d samples: %.2f sec' % (n_samples, time.time() - t0))

NameError: name 'cifar_X' is not defined

In [6]:
deepview.close()

## Using DeepView in a training routine

In [4]:
batch_size = 64
mnist_ds = demo.create_torch_dataset(batch_size, train=True)

In [8]:
# os.mkdir('results')
import umap

umap.UMAP()

umap.umap_.UMAP

In [7]:
model = demo.TorchModel()
model = model.to(device)
step = 0
epochs = 2
batches = np.ceil(len(mnist_ds)/batch_size)
# ----------- SETUP DeewView ----------------
title = 'ResNet-20 - CIFAR10'
max_samples = 200
pred_wrapper = demo.create_torch_wrapper(model, device)
verbose = False
classes = range(10)
data_shape = (28, 28, 1)

deepview = DeepView(pred_wrapper, classes, max_samples, batch_size, data_shape, 
                    n, lam, resolution, cmap, title=title, verbose=verbose, seed=42)

# add 4 batches of points to DeepView
ds = iter(mnist_ds)
for i in range(4):
    xs, ys = next(ds)
    xs = xs.transpose(3,1).transpose(2,1).cpu()
    deepview.add_samples(xs.numpy(), ys.cpu().numpy(), False)

In [8]:
deepview.lam = 0.75

In [9]:
for epoch in range(epochs):
    
    for i, (X, y) in enumerate(mnist_ds):
        X = X.to(device)
        y = y.to(device)
        model.optimizer.zero_grad()
        logits = model.forward(X)
        preds = softmax(logits)
        loss = model.loss_fn(preds, y)
        loss.backward()
        model.optimizer.step()
        step += 1
        
        if i % 10 == 0:
            # update samples in deepview
            deepview.update()
            title = 'cifar10 step %d loss %.2f' % (step, loss.cpu().detach().numpy())
            deepview.ax.set_title(title)
            deepview.fig.savefig('results/deepview_mnist_%d.pdf' % step)
            print(loss.cpu().detach().numpy())

2.3025665
2.2923768
2.1530974
2.0211568
1.8018416
1.7136234
1.6662513
1.6412671
1.582039
1.6288692
1.6654189
1.6106936
1.6682903
1.630942
1.5531472
1.5893736
1.6786302
1.6214026
1.6534883
1.5533822
1.5617626
1.5793964
1.5829853
1.5983434
1.5686294
1.585761
1.5563494
1.5375633
1.6427846
1.5694093
1.511967
1.5416645
1.5833257
1.5283006
1.5613956
1.5878615
1.5261532
1.5190861
1.5218796
1.5560772
1.539955
1.5708776
1.6229061
1.5614201
1.6225318
1.5302511
1.5158979
1.4761128
1.5084955
1.5335644
1.5551655
1.5447632
1.488345
1.5116361
1.4964716
1.6167387
1.5570909
1.5491014
1.6043891
1.5163338
1.5476564
1.5622323
1.5220479
1.5339631
1.504228
1.5358031
1.5278969
1.5016149
1.5216564
1.5175843
1.5115942
1.4951671
1.5148455
1.5200933
1.5549532
1.5814438
1.5024128
1.504522
1.5372555
1.50006
1.4961711
1.4930007
1.5415187
1.5301611
1.5267262
1.4915701
1.5503973
1.5046484
1.4890081
1.4960154
1.5343136
1.4859928
1.4970052
1.5481327
1.5186193
1.4929318
1.5367404
1.4923218
1.4664534
1.5321945
1.5046794


In [ ]:
-> new -> old

In [165]:
old_ebd = deepview.embedded

In [167]:
new_ebd = deepview.embedded

In [168]:
print(new_ebd[:5])
old_ebd[:5]

[[-1.909006    2.233761  ]
 [ 2.2817965   0.01644166]
 [-0.8881552   1.1460644 ]
 [-0.24402298  3.876659  ]
 [-1.3976914   1.9913878 ]]


array([[-2.1311893 ,  2.455944  ],
       [ 2.368148  ,  0.10293511],
       [-0.7888781 ,  1.0468191 ],
       [-0.20028041,  3.8314996 ],
       [-1.3965527 ,  2.0034225 ]], dtype=float32)

In [36]:
x_min = old_ebd.min(0)
x_max = old_ebd.max(0)
av_range = np.mean(x_max - x_min)

a = 500/av_range
b = 1

In [166]:
deepview.mapper.random_state = np.random.randint(1000)
deepview.update()

In [142]:
import umap
import matplotlib.pyplot as plt

new_eb = umap.umap_.optimize_layout_euclidean(
    new_ebd.copy(), old_ebd.copy(), list(range(deepview.num_samples)), list(range(deepview.num_samples)), 
    100, deepview.num_samples, np.array([1]*deepview.num_samples), a, b, np.array([1,1,1]))

In [145]:
print(new_eb[:5])
old_ebd[:5]

[[-1.3052449  8.849721 ]
 [ 1.1837322  8.686272 ]
 [ 2.199946   6.4908895]
 [ 3.6811585  4.8167205]
 [ 1.5350261  4.4847336]]


array([[-3.9419956, 12.466264 ],
       [ 2.771194 ,  7.8540854],
       [ 3.297338 ,  7.233462 ],
       [ 5.372856 ,  4.2202163],
       [ 2.0815883,  4.518349 ]], dtype=float32)

In [30]:
fig, ax = plt.subplots(2, 2)

ax[0,0].scatter(new_eb)
ax[0,1].scatter(new_eb)
ax[1,0].scatter(new_eb)
ax[1,1].scatter(new_eb)

In [32]:
for i in range(20):
    #if int((2)**i % 20) == 0:
    #    print(i)
    print(i, np.log(i+1), int(2**i))

0 0.0 1
1 0.6931471805599453 2
2 1.0986122886681098 4
3 1.3862943611198906 8
4 1.6094379124341003 16
5 1.791759469228055 32
6 1.9459101490553132 64
7 2.0794415416798357 128
8 2.1972245773362196 256
9 2.302585092994046 512
10 2.3978952727983707 1024
11 2.4849066497880004 2048
12 2.5649493574615367 4096
13 2.6390573296152584 8192
14 2.70805020110221 16384
15 2.772588722239781 32768
16 2.833213344056216 65536
17 2.8903717578961645 131072
18 2.9444389791664403 262144
19 2.995732273553991 524288


## Demo with RandomForest

In [27]:
pred_wrapper = DeepView.create_simple_wrapper(random_forest.predict_proba)

# the digit dataset is used, so classes are [0..9]
classes = np.arange(10)

# --- Deep View Parameters ----
batch_size = 64
max_samples = 500
sample_shape = (64,)
n = 10
lam = 0.5
resolution = 100
cmap = 'tab10'
title = 'RandomForest - MNIST'

# create DeepView object
deepview = DeepView(pred_wrapper, classes, max_samples, batch_size, 
                    sample_shape, n, lam, resolution, cmap, title=title)

# add data samples
n_samples = 200
sample_ids = np.random.choice(len(digits_X), n_samples)
X = np.array([ digits_X[i] for i in sample_ids ])
Y = np.array([ digits_y[i] for i in sample_ids ])

t0 = time.time()
deepview.add_samples(X, Y)
deepview.show()

print('Time to calculate visualization for %d samples: %.2f sec' % (n_samples, time.time() - t0))

Distance calculation 20.00 %
Distance calculation 40.00 %
Distance calculation 60.00 %
Distance calculation 80.00 %
Distance calculation 100.00 %
Embedding samples ...


/home/luca/Downloads/umap/umap/umap_.py:1428: UserWarning: Using precomputed metric; transform will be unavailable for new data
  warn("Using precomputed metric; transform will be unavailable for new data")


Computing decision regions ...
Time to calculate visualization for 200 samples: 65.11 sec


![random_forest](https://user-images.githubusercontent.com/30961397/78502477-a6ab5200-7761-11ea-8be3-e0b4c8e6a966.png)

In [28]:
deepview.close()

## Demo with DecisionTree

In [29]:
# --- Deep View Parameters ----
batch_size = 256
max_samples = 500
# the data can also be represented as a vector
sample_shape = (64,)
n = 10
lam = 0.65
resolution = 100
cmap = 'gist_ncar'

# the digit dataset is used, so classes are [0..9]
classes = np.arange(10)

In [30]:
pred_wrapper = DeepView.create_simple_wrapper(kn_neighbors.predict_proba)

# create DeepView object
deepview = DeepView(pred_wrapper, classes, max_samples, batch_size, 
                    sample_shape, n, lam, resolution, cmap)

# add data samples
n_samples = 200
sample_ids = np.random.choice(len(digits_X), n_samples)
X = np.array([ digits_X[i] for i in sample_ids ])
Y = np.array([ digits_y[i] for i in sample_ids ])

t0 = time.time()
deepview.add_samples(X, Y)
deepview.show()

print('Time to calculate visualization for %d samples: %.2f sec' % (n_samples, time.time() - t0))

Distance calculation 20.00 %
Distance calculation 40.00 %
Distance calculation 60.00 %
Distance calculation 80.00 %
Distance calculation 100.00 %
Embedding samples ...


/home/luca/Downloads/umap/umap/umap_.py:1428: UserWarning: Using precomputed metric; transform will be unavailable for new data
  warn("Using precomputed metric; transform will be unavailable for new data")


Computing decision regions ...
Time to calculate visualization for 200 samples: 77.18 sec


In [13]:
deepview.set_lambda(.4)
deepview.show()

Embedding samples ...


/home/luca/Downloads/umap/umap/umap_.py:1428: UserWarning: Using precomputed metric; transform will be unavailable for new data
  warn("Using precomputed metric; transform will be unavailable for new data")


Computing decision regions ...


In [9]:
deepview.close()

## Demo: KNN-Classifier

In [31]:
pred_wrapper = DeepView.create_simple_wrapper(kn_neighbors.predict_proba)

# create DeepView object
deepview = DeepView(pred_wrapper, classes, max_samples, batch_size, 
                    sample_shape, n, lam, resolution, cmap)

# add data samples
n_samples = 200
sample_ids = np.random.choice(len(digits_X), n_samples)
X = np.array([ digits_X[i] for i in sample_ids ])
Y = np.array([ digits_y[i] for i in sample_ids ])

t0 = time.time()
deepview.add_samples(X, Y)
deepview.show()

print('Time to calculate visualization for %d samples: %.2f sec' % (n_samples, time.time() - t0))

Distance calculation 20.00 %
Distance calculation 40.00 %
Distance calculation 60.00 %
Distance calculation 80.00 %
Distance calculation 100.00 %
Embedding samples ...


/home/luca/Downloads/umap/umap/umap_.py:1428: UserWarning: Using precomputed metric; transform will be unavailable for new data
  warn("Using precomputed metric; transform will be unavailable for new data")


Computing decision regions ...
Time to calculate visualization for 200 samples: 79.15 sec


![knn](https://user-images.githubusercontent.com/30961397/78502740-dc046f80-7762-11ea-82cf-efc8251539db.png)


## Evaluating DeepView

In [12]:
from deepview.evaluate import evaluate

print('Evaluation of DeepView: %s\n' % deepview.title)
evaluate(deepview, X, Y)

Evaluation of DeepView: ResNet-20 - CIFAR10

orig labs, knn err: eucl / fish 0.29 / 0.46
orig labs, knn err in proj space: eucl / fish 0.8 / 0.29
classif labs, knn err: eucl / fish 0.26 / 0.5
classif labs, knn acc in proj space: eucl / fish 17.0 / 74.0


In [ ]:
deepview.close()